In [98]:
# import libaries
import git
import dvc.api
import pandas as pd
import mlflow

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import mlflow
import mlflow.sklearn

In [73]:
# functions

# print repository information
def print_repository_info(repo):
    print('Repository description: {}'.format(repo.description))
    print('Repository active branch is {}'.format(repo.active_branch))

    for remote in repo.remotes:
        print('Remote named "{}" with URL "{}"'.format(remote, remote.url))

    print('Last commit for repository is {}.'.format(str(repo.head.commit.hexsha)))

#get required commit:
def get_commit_data(tagName):
    tags = repo.tags
    for tag in tags:
        if tag.name.strip("'") == tagName:
            commit = repo.commit(tag)
    print('-----------------------------')
    print(str(commit.hexsha))
    print("\"{}\" by {} ({})".format(commit.summary, commit.author.name, commit.author.email))
    print(str(commit.authored_datetime))
    print(str("count: {} and size: {}".format(commit.count(), commit.size)))
    # get the required version
    return(str(commit.hexsha))

In [75]:
repo_path = r"D:\ITI-AI-PRO\Production\Iris_project_mlflow\project\.git"
repo = git.Repo(repo_path)

In [103]:
def train(repo_path=repo_path, data_tag = 'v1', model_Name='knn', p1_value=5):
        ############################################# Load Repo #############################################
    repo = git.Repo(repo_path)
    # check that the repository loaded correctly
    if not repo.bare:
        print('Repo successfully loaded.')
        print_repository_info(repo)

        ############################################# Get the data #############################################
        # get the required commit
        rev1 = get_commit_data(data_tag)

        resource_url = dvc.api.get_url('data/Iris.csv',
                                        repo = repo_path,
                                        rev = rev1
        )
        print('-----------------------------')
        print(resource_url)

        data = pd.read_csv(resource_url2, sep=',')
        if data_tag == 'v1':
            data = data.drop(columns=['Id'])
        print(len(data))
        display(data)

        ############################################# MODEL #############################################
        #Create Training and Test Sets and Apply Scaling

        features= ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

        def create_75_25_split():
            X_train, X_test, y_train, y_test = train_test_split(data[features], data.Species, random_state=42)
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            return X_train, X_test, y_train, y_test
        # Split the data
        X_train, X_test, y_train, y_test =create_75_25_split()

        # Choose the model
        if model_Name == 'knn':
            p1 ='K'
            knn = KNeighborsClassifier(n_neighbors=p1_value)
            model = knn.fit(X_train, y_train)
        elif model_Name == 'logreg':
            p1 ='C'
            logreg = LogisticRegression(C=p1_value)
            model = logreg.fit(X_train, y_train)
        elif model_Name == 'RandomForest':
            p1 ='max_depth'
            rf = RandomForestClassifier(max_depth=p1_value)
            model = rf.fit(X_train, y_train)
        # Make predictions
        y_pred = model.predict(X_test)
        # Get metrics
        train_accuracy = model.score(X_train, y_train)
        test_accuracy = model.score(X_test, y_test)
        f1_score_test = f1_score(y_test, y_pred, average='micro')
        print('Accuracy of {} classifier on training set: {:.2f}'.format(model_Name, train_accuracy))
        print('Accuracy of {} classifier on test set: {:.2f}'.format(model_Name, test_accuracy))
        

        ############################################# Tracking #############################################
        # set to your server URI
        remote_server_uri = "http://127.0.0.1:5000"
        mlflow.set_tracking_uri(remote_server_uri)

        # Run mlflow
        mlflow.set_experiment("/Iris__Tracking")

        with mlflow.start_run():

            # data tracking
            mlflow.log_param("Data Tag", data_tag)
            mlflow.log_param("Data path", resource_url)
            mlflow.log_param("Data Count", len(data))

            # Model tracking
            mlflow.sklearn.log_model(model, "model")

            # model parameters tracking
            mlflow.log_param(f"{p1}", p1_value)
            mlflow.log_param("Model Name", model_Name)

            # model metrics tracking
            mlflow.log_metric("train_accuracy", train_accuracy)
            mlflow.log_metric("test_accuracy", test_accuracy)
            mlflow.log_metric("f1_score_test", f1_score_test)


    else:
        print('Could not load repository at {} :'.format(repo_path))

In [100]:
train(repo_path=repo_path, data_tag = 'v1', model_Name='knn', p1_value=5)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of knn classifier on training set: 0.97
Accuracy of knn classifier on test set: 0.92


In [101]:
train(repo_path=repo_path, data_tag = 'v2', model_Name='knn', p1_value=7)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of knn classifier on training set: 0.97
Accuracy of knn classifier on test set: 0.92


In [104]:
train(repo_path=repo_path, data_tag = 'v2', model_Name='logreg', p1_value=1)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of logreg classifier on training set: 0.93
Accuracy of logreg classifier on test set: 0.86


In [105]:
train(repo_path=repo_path, data_tag = 'v2', model_Name='logreg', p1_value=2)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of logreg classifier on training set: 0.95
Accuracy of logreg classifier on test set: 0.89


In [106]:
train(repo_path=repo_path, data_tag = 'v2', model_Name='logreg', p1_value=0.5)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of logreg classifier on training set: 0.92
Accuracy of logreg classifier on test set: 0.86


In [107]:
train(repo_path=repo_path, data_tag = 'v1', model_Name='logreg', p1_value=0.5)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of logreg classifier on training set: 0.92
Accuracy of logreg classifier on test set: 0.86


In [108]:
train(repo_path=repo_path, data_tag = 'v1', model_Name='RandomForest', p1_value=10)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of RandomForest classifier on training set: 1.00
Accuracy of RandomForest classifier on test set: 0.92


In [109]:
train(repo_path=repo_path, data_tag = 'v2', model_Name='RandomForest', p1_value=5)

Repo successfully loaded.
Repository description: Unnamed repository; edit this file 'description' to name the repository.
Repository active branch is master
Last commit for repository is 5b7763e7bd66b98c58370284c51bdd855330e923.
-----------------------------
5b7763e7bd66b98c58370284c51bdd855330e923
"data: track" by Ahmed Nabil Awaad (ah.n.awaad@gmail.com)
2021-12-06 22:59:47+02:00
count: 2 and size: 240
-----------------------------
..\remote\ae\d7d8b896a0f613effbcb60a4164257
143


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
138,6.7,3.0,5.2,2.3,Iris-virginica
139,6.3,2.5,5.0,1.9,Iris-virginica
140,6.5,3.0,5.2,2.0,Iris-virginica
141,6.2,3.4,5.4,2.3,Iris-virginica


Accuracy of RandomForest classifier on training set: 1.00
Accuracy of RandomForest classifier on test set: 0.92
